In [32]:
from api_key import KEY
from urllib.parse import urlencode
import requests
import json
from datetime import datetime
import tqdm
import time
import pandas as pd 

In [2]:
# KEY

In [3]:
import yapi

# Identificators

In [4]:
PLAYLIST_ID = 'PLQPfsdgAXeMlB0vNrePI1qfhjSt91yShn'

# yAPI instance

In [5]:
api = yapi.YoutubeAPI(KEY)

# Fetching playlist items

In [6]:
# videos = vars(playlistItems)

In [7]:
# len(videos)

In [8]:
# videos.keys()

In [9]:
# len(videos["items"])

In [10]:
# videos['nextPageToken']

In [11]:
# playlistItems = api.get_playlist_items_by_playlist_id(PLAYLIST_ID, max_results=20, pageToken='CDIQAA')

In [12]:
# vars(videos["items"][0]).keys()

In [13]:
# vars(videos["items"][0])["contentDetails"]

In [14]:
def handle_response(url, params_dict):
    """ Sends request with specified params, returns list of items and nextPageToken
    """
    url = url+"?"+urlencode(params_dict)
    resp = requests.get(url)
#     print(resp)
    response_dict = json.loads(resp.text)
    if "nextPageToken" in response_dict:
        return_dict = {
            "items": response_dict["items"],
            "nextPageToken": response_dict["nextPageToken"]
        }
    else: 
        return_dict = {
            "items": response_dict["items"],
            "nextPageToken": None
        }
    return return_dict

In [15]:
def get_all_playlist_items(params_dict):
    """ Returns all video ID's from specified Play List
    """
    item_ids = []
    stop = False
    while stop == False:
        url = "https://www.googleapis.com/youtube/v3/playlistItems"
        if item_ids == []:
            response = handle_response(url, params_dict)
            for item in response["items"]:
                item_ids.append(item["contentDetails"]['videoId'])
#                 print(item["contentDetails"]['videoId'])
            npt = response["nextPageToken"]
            print("FIRST NPT: {}".format(npt))
        else: 
            params_dict["pageToken"] = npt
            response = handle_response(url, params_dict)
            for item in response["items"]:
                item_ids.append(item["contentDetails"]['videoId'])
            npt = response["nextPageToken"]
            print("NPT: {}".format(npt))
            print(len(response["items"]))
            if npt == None:
                stop = True
    return item_ids

In [16]:
params = {
            'playlistId': PLAYLIST_ID,
            'part': 'id, snippet, contentDetails, status',
            'maxResults': 50,
            'key': KEY,
        }
video_ids = get_all_playlist_items(params)

FIRST NPT: CDIQAA
NPT: CGQQAA
50
NPT: CJYBEAA
50
NPT: CMgBEAA
50
NPT: CPoBEAA
50
NPT: CKwCEAA
50
NPT: CN4CEAA
50
NPT: CJADEAA
50
NPT: CMIDEAA
50
NPT: None
25


# Fetching video meta data

In [17]:
# video = api.get_video_info('xsSz-l8G-0Q')

In [18]:
# vars(video).keys()

In [19]:
# vars(video)

In [20]:
# vars(vars(video)["items"][0])

In [21]:
# url = "https://www.googleapis.com/youtube/v3/videos" 
# params = {
#             'id': video_ids[0],
#             'part': 'id, snippet, contentDetails, player, statistics, status',
#             'key': KEY,
#         }
# url = url + "?" + urlencode(params)
# resp = requests.get(url)

In [22]:
# response_dict = json.loads(resp.text)
# print(response_dict.keys())
# print(response_dict["kind"])
# print(response_dict["pageInfo"])
# print(response_dict["items"][0].keys())

In [23]:
# response_dict["items"][0]["statistics"]

In [24]:
# response_dict["items"][0]["snippet"].keys()

In [25]:
# response_dict["items"][0]["snippet"]["publishedAt"]

In [26]:
# response_dict["items"][0]["snippet"]["title"]


In [27]:
# response_dict["items"][0]["snippet"]["description"]

In [28]:
# vars(vars(video)["items"][0])["statistics"]

In [29]:
# response_dict["items"][0]["snippet"]["tags"]

In [35]:
def save_video_metadata(key, video_ids, path_pickle, path_responses):
    """ Returns a dataframe with metadata and saves all responses in a json file 
    """
#     print(key)
    resp_lst = []
    endpoint = "https://www.googleapis.com/youtube/v3/videos" 
    metadata_lst = []
    for video_id in tqdm.tqdm(video_ids):
        params = {
                    'id': video_id,
                    'part': 'id, snippet, contentDetails, player, statistics, status',
                    'key': key,
                }
#         print(params)
        url = endpoint + "?" + urlencode(params)
#         print(url)
#         time.sleep(0.5)
        resp = requests.get(url)
        response_dict = json.loads(resp.text)
        resp_lst.append(response_dict)
#         print(response_dict.keys())
        title = response_dict["items"][0]["snippet"]["title"]
        stats = response_dict["items"][0]["statistics"]
        comments = stats['commentCount']
        dislikes = stats['dislikeCount']
        favourites = stats['favoriteCount']
        likes = stats['likeCount']
        views = stats['viewCount']
        tags = response_dict["items"][0]["snippet"]["tags"]
        published = datetime.strptime(response_dict["items"][0]["snippet"]["publishedAt"].split("T")[0], '%Y-%m-%d')
        sub_lst = [title, published, tags, comments, likes, dislikes, favourites, views]
        metadata_lst.append(sub_lst)
    df = pd.DataFrame(metadata_lst)
    df.columns = ["title", "published", "tags", "comments", "likes", "dislikes", "favourites", "views"]
    df.to_pickle(path_pickle)
    print("Pickle saved at {}".format(path_pickle))
    with open(path_responses, 'w') as outfile:
        json.dump(resp_lst, outfile)
    print("Responses saved at {}".format(path_responses))

In [36]:
save_video_metadata(KEY, video_ids, "../data/videos", "../data/videos_raw.json")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 475/475 [01:55<00:00,  4.25it/s]


Pickle saved at ../data/videos
Responses saved at ../data/videos_raw.json


In [40]:
# video_ids

In [39]:
# pd.read_json("../data/json")